In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [2]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1486732596.log


This non-commercial license of GraphLab Create for academic use is assigned to herreramachado@gmail.com and will expire on January 24, 2018.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 2.137591     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 2.206060     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [81]:
model_all.get('coefficients').print_rows(num_rows=30, num_columns=50)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [94]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [86]:
import numpy as np
from decimal import Decimal

my_range = np.logspace(1, 7, num=13)
min_err = 1e100000000000000
min_l1_pen = 0

for l1_penalty_new in my_range:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                             validation_set = None, 
                                             l2_penalty = 0., l1_penalty = l1_penalty_new, verbose = False)
    prediction = model.predict(validation[all_features])
    error = validation['price'] - prediction
    RSS = sum(error ** 2)
    if(RSS < min_err):
        min_err = RSS
        min_l1_pen = l1_penalty_new
print min_err, Decimal(min_l1_pen)

6.25766285142e+14 10


In [95]:
import numpy as np
from decimal import Decimal

my_range = np.logspace(1, 7, num=13)

for l1_value in my_range:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                             validation_set = None, 
                                             l1_penalty = l1_value, l2_penalty = 0., verbose = False)
    print(model.get('coefficients')['value'])

[18993.427212770577, 7936.967679031306, 936.993368193299, 25409.58893412067, 39.11513637970762, 1124.6502128077207, 0.0034836182229897442, 148.2583910114082, 21204.33546695013, 12915.524336072436, 601905.5945452718, 93312.85731187189, 6609.035712447216, 6206.939991880551, 43.28705341933558, 122.36782753411931, 9.433635393724884, 56.072003448822386]
[18993.42853454867, 7936.967648008103, 936.9933489896162, 25409.588897696016, 39.11513636493602, 1124.6502112987225, 0.0034836054902507795, 148.25839014793078, 21204.335358875258, 12915.524239891067, 601905.5872644086, 93312.856828489, 6609.035659704598, 6206.939977681307, 43.287053366889275, 122.36782679765587, 9.433635310390496, 56.0720018382073]
[18993.432714378352, 7936.967549904135, 936.9932882622419, 25409.588782511095, 39.11513631822403, 1124.6502065268521, 0.003483565225795815, 148.25838741737542, 21204.33501711258, 12915.523935738885, 601905.5642402971, 93312.85529989799, 6609.0354929178275, 6206.939932779336, 43.28705320103935, 122

In [73]:
'%.2E' % Decimal(100000)
print(1.00E+01)

10.0


In [30]:
6.25766285142e+14 10.0

SyntaxError: invalid syntax (<ipython-input-30-751d06d3a1f4>, line 1)

In [45]:
print 1e7
print 10**7

10000000.0
10000000


In [97]:
weights = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                             validation_set = None, 
                                             l2_penalty = 0., l1_penalty = 10, verbose = False)
weights.get('coefficients')['value'].nnz()

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [46]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [48]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [55]:
import numpy as np
from decimal import Decimal

my_range = l1_penalty_values
min_err = 1e100000000000000
min_l1_pen = 0
nonzeros = []

for l1_penalty_new in my_range:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                             validation_set = None, 
                                             l2_penalty = 0., l1_penalty = l1_penalty_new, verbose = False)
    
    nonzeros.append(model.get('coefficients')['value'].nnz())
print(nonzeros)

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [56]:
l1_penalty_min = l1_penalty_values[14]
l1_penalty_max = l1_penalty_values[16]
print(l1_penalty_min, l1_penalty_max)

(2976351441.6313133, 4832930238.5717525)


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [60]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [98]:
import numpy as np
from decimal import Decimal

my_range = l1_penalty_values
min_err = 1e100000000000000
min_l1_pen = 0

for l1_penalty_new in my_range:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                             validation_set = None, 
                                             l2_penalty = 0., l1_penalty = l1_penalty_new, verbose = False)
    prediction = model.predict(validation[all_features])
    error = validation['price'] - prediction
    RSS = sum(error ** 2)
    non_zero = model.get('coefficients')['value'].nnz()
    if(RSS < min_err and non_zero == max_nonzeros):
        min_err = RSS
        min_l1_pen = l1_penalty_new
        features = model.get('coefficients')['value', 'name']
print Decimal(min_err), Decimal(min_l1_pen)

1048315056620935.625 3464924809.24721813201904296875


In [69]:
features.print_rows(num_rows=20, num_columns=2)

+---------------+------------------+
|     value     |       name       |
+---------------+------------------+
| 223113.283157 |   (intercept)    |
|  603.81178122 |     bedrooms     |
|      0.0      | bedrooms_square  |
| 15791.8297397 |    bathrooms     |
|  32.338436177 |   sqft_living    |
| 686.175362908 | sqft_living_sqrt |
|      0.0      |     sqft_lot     |
|      0.0      |  sqft_lot_sqrt   |
|      0.0      |      floors      |
|      0.0      |  floors_square   |
|      0.0      |    waterfront    |
|      0.0      |       view       |
|      0.0      |    condition     |
| 2873.14181982 |      grade       |
| 29.9057289304 |    sqft_above    |
|      0.0      |  sqft_basement   |
|      0.0      |     yr_built     |
|      0.0      |   yr_renovated   |
+---------------+------------------+
[18 rows x 2 columns]

